In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import xgboost as xgb
from params import col_nuls,col_less_10, RANDOM_STATE, params_xgb


/home/prometheus/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
scores_train = pd.read_pickle('data/stacking/scores_train.pkl', compression='gzip')
scores_test = pd.read_pickle('data/stacking/scores_test.pkl', compression='gzip')

In [3]:
scores_train_lgb = pd.read_pickle('data/stacking/scores_train_lgb.pkl', compression='gzip')
scores_test_lgb = pd.read_pickle('data/stacking/scores_test_lgb.pkl', compression='gzip')

In [7]:
target = pd.read_pickle('data/drop_duplicates/target.pkl', compression='gzip')

In [8]:
scores_train = pd.merge(scores_train,scores_train_lgb,how='inner',right_index=True,left_index=True)

In [9]:
xgb_train = xgb.DMatrix(scores_train, label=target)
cv_res = xgb.cv(params_xgb, xgb_train, num_boost_round=500, nfold=5, stratified=True, )
num_boost_best = np.argmax(cv_res['test-auc-mean'])
print np.argmax(cv_res['test-auc-mean']), np.max(cv_res['test-auc-mean'])

197 0.7013552


In [10]:
scores_train[:1]

,xgb_predict_x,xgb_predict_y
ID,,
24368,0.034852,0.050091


In [15]:
from sklearn.metrics import roc_auc_score

print roc_auc_score(target, scores_train['xgb_predict_x']),roc_auc_score(target, scores_train['xgb_predict_y'])

0.701910970121 0.687699851368


In [19]:
scores_train['avg'] = (scores_train['xgb_predict_x'].rank()+scores_train['xgb_predict_y'].rank())/2

In [20]:
print roc_auc_score(target, scores_train['avg'])

0.698846123467
